---
### Aluno: Yann Bruno Andrade Mello
### Engenharia Elétrica - UFC
---

# Bibliotecas Utilizadas

In [1]:
import numpy as np  # Usada para operar matrizes
import random      # Usada para gerar valores aleatórios
import pandas as pd  # Usada para importar a base de dados .xlsx
from neurolab.trans import HardLim  # Usada para chamar a função hardlim do MATLAB

# Definindo o perceptron

In [2]:
def perceptron(x, y, w0, eta, max_epoca):
    wf = [0]*len(w0)  # Vetor definido para guardar os pesos finais
    epoca = 0  # Setando a quantidade de épocas
    # Definindo um vetor para guardar os erros
    erro = np.array([1 for n in range(len(y))])
    while epoca <= max_epoca:
        for n in range(len(y)):
            """
            u é o potencial de ativação
            func recebe a função HardLim
            ativa recebe o valor da função de aivação no potencial de ativação

            """
            u = w0.T@x.T[n]
            func = HardLim()
            ativa = func(u)

            # Armazena o erro das respectivas entradas e respectiva saídas
            erro[n] = y[n] - ativa

            while ativa != y[n]:  # Laço de tratamento dos pesos
                wf = w0 + eta*(y[n]-ativa)*(x.T[n])
                u_n = wf.T@x.T[n]
                ativa_n = func(u_n)
                w0 = wf
                if ativa_n == y[n]:
                    ativa = ativa_n

            wf = w0

        if np.all(erro == 0):  # Checa se todos os erros são 0
            break
        else:
            epoca += 1  # iterador de época

    return [wf, epoca]

# Definindo a operação do Perceptron

In [3]:
def operacao(wf, amostra):
    func = HardLim()
    retorno = func(wf.T@amostra)
    if retorno == 0:
        retorno = -1
    return retorno

---

# Aplicação no Projeto Prático ([1] - páginas: 70-72)

## Import da base de dados com valores para teste
Houve um tratamento dos dados de entrada:
- Foi usado `replace()` para mudar os valores -1, da saída, para 0, para facilitar a função de ativação
- Após isso fizemos a transposição do DataFrame, afim de facilitar quando convertermos para um array numpy

In [4]:
df = ((pd.read_excel('Base.xlsx')).replace(-1, 0)).T
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
x1,-0.6508,-1.4492,2.0850,0.2626,0.6418,0.2569,1.1155,0.0914,0.0121,-0.0429,...,-0.1147,-0.7970,-1.0625,0.5307,-1.2200,0.3957,-0.1013,2.4482,2.0149,0.2012
x2,0.1097,0.8896,0.6876,1.1476,1.0234,0.6730,0.6043,0.3399,0.5256,0.4660,...,0.2242,0.8795,0.6366,0.1285,0.7777,0.1076,0.5989,0.9455,0.6192,0.2611
x3,4.0009,4.4005,12.0710,7.7985,7.0427,8.3265,7.4446,7.0677,4.6316,5.4323,...,7.2435,3.8762,2.4707,5.6883,1.7252,5.6623,7.1812,11.2095,10.9263,5.4631
d,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,1.0000,0.0000,1.0000,1.0000,...,0.0000,1.0000,1.0000,1.0000,1.0000,0.0000,0.0000,1.0000,0.0000,1.0000


## Convertendo o DataFrame para um array
- Para os valores de y, pegamos a última linha da Dataframe e armazenamos em uma array
- Para os valores das amostras de treinamento, foi usado o restante do Dataframe, convertido também em um array

In [5]:
y = df.to_numpy()[-1]  # Valores de saída

In [6]:
x = np.delete(df.to_numpy(), -1, 0)
# Definição dos valores de entrada
amostra_treinamento = np.insert(x, 0, (np.array([-1]*len(y))), axis=0)

---

## Definição dos pesos iniciais $w_0$
Instruções do livro:
1) Execute cinco treinamentos para a rede Perceptron, iniciando-se o vetor de pesos {w} em cada treinamento com valores aleatórios entre zero e um. Se for o caso, reinicie o gerador de números aleatórios em cada treinamento de tal forma que os elementos do vetor de pesos iniciais não sejam os mesmos. O conjunto de treinamento encontra-se no apêndice I.<br />
<br />
2) Registre os resultados dos cinco treinamentos na tabela

In [7]:
conjunto_pesos_i = []  # Vetor usado para guardar os valores dos pesos iniciais
for n in range(5):
    # Gera valores entre 0 e 1 aleatórios para os pesos iniciais
    conjunto_pesos_i.append(np.concatenate(
        ([0], [random.random() for i in range(3)])))
# Transforma o array de pesos iniciais em um Dataframe
pesos = pd.DataFrame(conjunto_pesos_i, index=[
                     '1º(T1)', '2º(T2)', '3º(T3)', '4º(T4)', '5º(T5)'], columns=['w0 = eta', 'w1', 'w2', 'w3'])
display(pesos.style.set_caption(
    'Tabela 3.2 – Resultados dos treinamentos do Perceptron - Parte 1'))

,w0 = eta,w1,w2,w3
1º(T1),0.000000,0.199186,0.658939,0.132476
2º(T2),0.000000,0.674436,0.073872,0.937378
3º(T3),0.000000,0.205966,0.709620,0.859138
4º(T4),0.000000,0.323751,0.476434,0.076662
5º(T5),0.000000,0.190562,0.510284,0.815919


### Plotagem da tabela de pesos iniciais (Tabela 3.2 - Resultados dos treinamentos do Perceptron - Parte 1)

In [8]:
pesos

,w0 = eta,w1,w2,w3
1º(T1),0.0,0.199186,0.658939,0.132476
2º(T2),0.0,0.674436,0.073872,0.937378
3º(T3),0.0,0.205966,0.709620,0.859138
4º(T4),0.0,0.323751,0.476434,0.076662
5º(T5),0.0,0.190562,0.510284,0.815919


---

## Definição da taxa de aprendizagem

In [9]:
taxa_aprendizagem = 0.01

## Definição da quantidade máxima de épocas 

In [10]:
max_epoca = 10000

---

# Definição dos pesos finais e a quantidade de épocas, após usar o Perceptron

In [11]:
conjunto_pesos_f = [perceptron(
    amostra_treinamento, y, pesos.iloc[n], taxa_aprendizagem, max_epoca)[0] for n in range(5)]
# Aplicação do algoritmo de treinamento do Perceptron e armazenando o valor dos novos pesos em um array
conjunto_pesos_f = pd.DataFrame(conjunto_pesos_f)
# Construindo a Tabela 3.2 (inserindo a quantidade de épocas de cada treinamento)
conjunto_pesos_f_epocas = conjunto_pesos_f.assign(epocas=[perceptron(
    amostra_treinamento, y, pesos.iloc[n], taxa_aprendizagem, max_epoca)[1] for n in range(5)])

### Plotagem da tabela de pesos finais (Tabela 3.2 - Resultados dos treinamentos do Perceptron - Parte 2)

In [12]:
conjunto_pesos_f_epocas

,w0 = eta,w1,w2,w3,epocas
1º(T1),-1.53,0.772820,1.201581,-0.361756,392
2º(T2),-1.52,0.772486,1.214734,-0.361614,402
3º(T3),-1.57,0.787115,1.222087,-0.370337,431
4º(T4),-1.54,0.795747,1.211427,-0.365093,405
5º(T5),-1.58,0.769718,1.225910,-0.357487,440


---

# Colocando o Perceptron em operação, classificando os óleos da Tabela 3.3
3) Após o treinamento do Perceptron, coloque este em operação, aplicando-o na classificação automática das amostras de óleo da tabela 3.3, indicando ainda nesta tabela aqueles resultados das saídas (Classes) referentes aos cinco processos de treinamento realizados no item 1.

In [13]:
amostra = [[-1, -0.3665, 0.0620, 5.9891],
           [-1, -0.7842, 1.1267, 5.5912],
           [-1, 0.3012, 0.5611, 5.8234],
           [-1, 0.7757, 1.0648, 8.0677],
           [-1, 0.1570, 0.8028, 6.3040],
           [-1, -0.7014, 1.0316, 3.6005],
           [-1, 0.3748, 0.1536, 6.1537],
           [-1, -0.6920, 0.9404, 4.4058],
           [-1, -1.3970, 0.7141, 4.9263],
           [-1, -1.8842, -0.2805, 1.2548]]  # Conjunto de amostra da questão

In [14]:
# Criação de um vetor auxiliar para armazenar os valores de saídas
aux = [[], [], [], [], []]

In [15]:
for i in range(len(pesos)):
    for j in range(len(amostra)):
        # Realiza o algoritmo de operação do Perceptron e armazena no array
        aux[i].append(operacao(conjunto_pesos_f.iloc[i], amostra[j]))

In [16]:
tabela_amostra = pd.DataFrame(
    amostra, index=[n for n in range(1, 11)], columns=['x0', 'x1', 'x2', 'x3'])

tabela_saida = pd.DataFrame(aux, index=[
                            'y(T1)', 'y(T2)', 'y(T3)', 'y(T4)', 'y(T5)'], columns=[n for n in range(1, 11)])

# Transforma o array da célula anterior em um Dataframe
tabela_saida = tabela_saida.T

In [17]:
tabela_final = pd.concat([tabela_amostra, tabela_saida], axis=1)

### Plotagem da tabela de saídas (Tabela 3.3 – Amostras de óleo para validar a rede Perceptron)

In [18]:
tabela_final

,x0,x1,x2,x3,y(T1),y(T2),y(T3),y(T4),y(T5)
1,-1,-0.3665,0.0620,5.9891,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1,-0.7842,1.1267,5.5912,1.0,1.0,1.0,1.0,1.0
3,-1,0.3012,0.5611,5.8234,1.0,1.0,1.0,1.0,1.0
4,-1,0.7757,1.0648,8.0677,1.0,1.0,1.0,1.0,1.0
5,-1,0.1570,0.8028,6.3040,1.0,1.0,1.0,1.0,1.0
6,-1,-0.7014,1.0316,3.6005,1.0,1.0,1.0,1.0,1.0
7,-1,0.3748,0.1536,6.1537,-1.0,-1.0,-1.0,-1.0,-1.0
8,-1,-0.6920,0.9404,4.4058,1.0,1.0,1.0,1.0,1.0
9,-1,-1.3970,0.7141,4.9263,-1.0,-1.0,-1.0,-1.0,-1.0
10,-1,-1.8842,-0.2805,1.2548,-1.0,-1.0,-1.0,-1.0,-1.0


---

# Demais questões

4) Explique por que o número de épocas de treinamento, em relação a esta aplicação, varia a cada vez que executamos o treinamento do Perceptron.


Resposta: 
- Pelo fato de se ter diferentes valores nos pesos de entrada, fazendo com que assim o ajuste necessário nos pesos mude para cada caso, consequentemente, como muda para cada caso, muda a quantidade de épocas necessária para cada caso

5) Para a aplicação em questão, discorra se é possível afirmar se as suas classes são linearmente separáveis.


Resposta:
- Podemos sim afirmar que as classes são linearmente separáveis, uma vez que o algoritmo do Perceptron convergiu para todos os casos

---

# Referência Bibliográfica
[1] REDE Perceptron. In: NUNES DA SILVA, Ivan; HERNANE SPATTI, Danilo; ANDRADE FLAUZINO, Rogério. Redes Neurais Artificiais para engenharia e ciências aplicadas. [S. l.: s. n.], 2019. cap. 3, p. 57-70.